In [301]:
import pandas as pd 
import numpy as np
import datetime as dt #installs necessary packages

path='/Users/caseycloutier/Desktop/Theses/Honors585 Paper Data/raw datasets' #sets directory to proper folder

ur20data=pd.read_csv('/Users/caseycloutier/Desktop/Theses/Honors585 Paper Data/raw datasets/UR2020.csv') #opens 2020 unemployment data file
ur20data['CountyName']=ur20data['County Name/State Abbreviation'].astype(str) #convert county names to strings
ur20data['Period']=ur20data['Period'].astype(str) #converts dates to strings

ur20data=ur20data[~ur20data['CountyName'].str.endswith('PR')] #removes entries from puerto rico
ur20data=ur20data[~ur20data['Period'].str.startswith('19')] #removes dates from 2019

ur20data['Period']=ur20data['Period'].replace(['20-Jan','20-Feb','20-Mar','20-Apr','20-May','20-Jun','20-Jul','20-Aug','20-Sep','20-Oct','20-Nov','Dec-20 p'],['2020-01-31','2020-02-29','2020-03-31','2020-04-30','2020-05-31','2020-06-30','2020-07-31','2020-08-31','2020-09-30','2020-10-31','2020-11-30','2020-12-31']) #changes dates to values reconizable by dt package
ur20data['Date']=pd.to_datetime(ur20data['Period'], errors='coerce')
ur20data['fips']=ur20data['LAUS Code'].astype(str)
ur20data['fips'] = [x[2:-8] for x in ur20data['fips']]

del ur20data['LAUS Code']
del ur20data['statefips']
del ur20data['countyfips']
del ur20data['County Name/State Abbreviation']
del ur20data['labor force']
del ur20data['Employed']
del ur20data['Unemployed']
del ur20data['Period'] #remove unneeded columns

In [302]:
ur21data=pd.read_csv(path+'/UR2021.csv')

ur21data['CountyName']=ur21data['County Name/State Abbreviation'].astype(str) #convert county names to strings
ur21data['Period']=ur21data['Period'].astype(str) #converts dates to strings

ur21data=ur21data[~ur21data['CountyName'].str.endswith('PR')] #removes entries from puerto rico
ur21data=ur21data[~ur21data['Period'].str.endswith('20')] #removes dates from 2020
ur21data['Period']=ur21data['Period'].replace(['Jan-21','Feb-21','Mar-21','Apr-21','May-21','Jun-21','Jul-21','Aug-21','Sep-21','Oct-21','Nov-21','Dec-21 p'],['2021-01-31','2021-02-28','2021-03-31','2021-04-30','2021-05-31','2021-06-30','2021-07-31','2021-08-31','2021-09-30','2021-10-31','2021-11-30','2021-12-31'])

ur21data['Date']=pd.to_datetime(ur21data['Period'], errors='coerce')


In [303]:

ur21data['fips']=ur21data['LAUS Code'].astype(str)
ur21data['fips'] = [x[2:-8] for x in ur21data['fips']]

del ur21data['LAUS Code']
del ur21data['statefips']
del ur21data['countyfips']
del ur21data['County Name/State Abbreviation']
del ur21data['labor orce']
del ur21data['Employed']
del ur21data['Unemployed']
del ur21data['Period'] #remove unneeded columns 

ur20data=ur20data.rename(columns={'unemployment_rate': 'UR_rate'}) #rename to match other df

frames=[ur20data, ur21data]
urdata=pd.concat(frames) #combines 2 UR df's


In [304]:
vaxdata=pd.read_csv(path+'/vaccinations2021.csv') #opens vaccine dataset
#footnotes here: https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh
vaxdata['Date']=vaxdata['Date'].astype(str)
vaxdata=vaxdata[~vaxdata['Date'].str.endswith('2022')]
vaxdata=vaxdata[~vaxdata['Recip_State'].str.startswith('PR')]
vaxdata['Date']=pd.to_datetime(vaxdata['Date'], errors='coerce')
vaxdata=vaxdata.sort_values(['Date'])

vaxdata=vaxdata[~vaxdata['Recip_State'].str.endswith('VI')]
vaxdata=vaxdata[~vaxdata['Recip_County'].str.startswith('Kalawao')]
vaxdata=vaxdata[~vaxdata['Recip_State'].str.endswith('GU')] #remove unneeded values

vaxdata=vaxdata.loc[(vaxdata['Date'] == '12/31/2020') |
                         (vaxdata['Date'] == '01/31/2021') | 
                         (vaxdata['Date'] == '02/28/2021') | (vaxdata['Date']=='03/31/2021') | (vaxdata['Date']=='04/30/2021') | (vaxdata['Date']=='05/31/2021') | (vaxdata['Date']=='06/30/2021') | (vaxdata['Date']=='07/31/2021') | (vaxdata['Date']=='08/31/2021') | (vaxdata['Date']=='09/30/2021') | (vaxdata['Date']=='10/31/2021') | (vaxdata['Date']=='11/30/2021') | (vaxdata['Date']=='12/31/2021')]
#filters df to needed rows
vaxdata=vaxdata[~vaxdata['FIPS'].str.endswith('UNK')] #removes data from unknown muni
vaxdata=vaxdata.rename(columns={'FIPS': 'fips'}) #rename column to match others


/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [305]:
urvaxdata=pd.merge(urdata, vaxdata,  how='left', left_on=['Date','fips'], right_on = ['Date','fips']) #merge UR and vax data

In [324]:
demodata=pd.read_csv('/Users/caseycloutier/Desktop/Theses/Honors585 Paper Data/raw datasets/county_demographics/demodata.csv')
demodata['GEO_ID'] = [x[9:] for x in demodata['GEO_ID']]
demodata=demodata.rename(columns={'GEO_ID': 'fips'})
demodata=demodata.filter(['fips','NAME','DP05_0002PE','DP05_0003PE','DP05_0037PE','DP05_0038PE','DP05_0039PE','DP05_0044PE','DP05_0052PE','DP05_0070PE'])
demodata=demodata.rename(columns={'GEO_ID': 'fips'})
demodata.drop([0], axis=0, inplace=True)

demodata=demodata[~demodata['NAME'].str.endswith('Puerto Rico')]
countynames=demodata['NAME'].str.split(', ', expand=True)
demodata=pd.concat([demodata, countynames], axis=1)

demodata['State']=demodata[1].replace({'Alabama':'AL', 'Alaska':'AK', 'Arizona':'AZ', 'Arkansas':'AR', 'California':'CA', 'Colorado':'CO', 'Connecticut':'CT', 'Delaware':'DE', 'Florida':'FL', 'Georgia':'GA', 'Hawaii':'HI', 'Idaho':'ID', 'Illinois':'IL', 'Indiana':'IN', 'Iowa':'IA', 'Kansas':'KS', 'Kentucky':'KY','Louisiana':'LA', 'Maine':'ME', 'Maryland':'MD', 'Massachusetts':'MA', 'Michigan':'MI', 'Minnesota':'MN', 'Mississippi':'MS', 'Missouri':'MO','Montana':'MT', 'Nebraska':'NE', 'Nevada':'NV', 'New Hampshire':'NH', 'New Jersey':'NJ', 'New Mexico':'NM', 'New York':'NY', 'North Carolina':'NC', 'North Dakota':'ND', 'Ohio':'OH', 'Oklahoma':'OK','Oregon':'OR','Pennsylvania':'PA','Rhode Island':'RI', 'South Carolina':'SC', 'South Dakota':'SD', 'Tennessee':'TN','Texas':'TX', 'Utah':'UT','Vermont':'VT', 'Virginia':'VA', 'Washington':'WA', 'West Virginia':'WV','Wisconsin':'WI', 'Wyoming':'WY'})

demodata['CountyName']=demodata[0]+', '+demodata['State']


In [325]:

del demodata['NAME']
del demodata[0]
del demodata[1]
del demodata['State']

demodata['CountyName']=demodata['CountyName'].astype(str)

demodata=demodata[~demodata['CountyName'].str.startswith('Kalawao')]

demodata2=demodata.append(demodata)
demodata3=demodata2.append(demodata)
demodata4=demodata3.append(demodata)
demodata5=demodata4.append(demodata)
demodata6=demodata5.append(demodata)
demodata7=demodata6.append(demodata)
demodata8=demodata7.append(demodata)
demodata9=demodata8.append(demodata)
demodata10=demodata9.append(demodata)
demodata11=demodata10.append(demodata)
demodata12=demodata11.append(demodata)
demodata13=demodata12.append(demodata)
demodata14=demodata13.append(demodata)
demodata15=demodata14.append(demodata)
demodata16=demodata15.append(demodata)
demodata17=demodata16.append(demodata)
demodata18=demodata17.append(demodata)
demodata19=demodata18.append(demodata)
demodata20=demodata19.append(demodata)
demodata21=demodata20.append(demodata)
demodata22=demodata21.append(demodata)
demodata23=demodata22.append(demodata)
demodata24=demodata23.append(demodata)

In [326]:
demodata=demodata24.sort_values('CountyName')
urvaxdata=urvaxdata.sort_values('CountyName')

In [327]:
date_col=urvaxdata.filter(['Date'])
date_col=date_col.reset_index()
demodata=demodata.reset_index()
demodata=pd.concat([date_col['Date'], demodata], axis=1)

In [328]:
demo_vax_ur=pd.merge(demodata, urvaxdata,  how='left', left_on=['Date','fips'], right_on = ['Date','fips'])
demo_vax_ur=demo_vax_ur.rename(columns={'DP05_0002PE': 'Percent_Male', 'DP05_0003PE': 'Percent_Female', 'DP05_0018E':'Median_Age','DP05_0037PE':'Percent_White','DP05_0038PE':'Percent_Black', 'DP05_0039PE':'Percent_AmerInd', 'DP05_0044PE':'Percent_Asian', 'DP05_0052PE':'Percent_PacI', 'DP05_0071PE':'Percent_HisLat'})
demo_vax_ur=demo_vax_ur.sort_values(['CountyName_x', 'Date'])
demo_vax_ur=demo_vax_ur[~demo_vax_ur['CountyName_x'].str.endswith('AK')]

In [329]:
#demo_vax_ur.to_csv('/Users/caseycloutier/Desktop/Theses/Honors585 Paper Data/cleaned data/demo_vax_ur.csv')

In [330]:
poli_data=pd.read_csv('/Users/caseycloutier/Desktop/Theses/Honors585 Paper Data/raw datasets/2020_US_County_Level_Presidential_Results.csv') #open political affiliation data

poli_data['state_name']=poli_data['state_name'].replace({'Alabama':'AL', 'Alaska':'AK', 'Arizona':'AZ', 'Arkansas':'AR', 'California':'CA', 'Colorado':'CO', 'Connecticut':'CT', 'Delaware':'DE', 'Florida':'FL', 'Georgia':'GA', 'Hawaii':'HI', 'Idaho':'ID', 'Illinois':'IL', 'Indiana':'IN', 'Iowa':'IA', 'Kansas':'KS', 'Kentucky':'KY','Louisiana':'LA', 'Maine':'ME', 'Maryland':'MD', 'Massachusetts':'MA', 'Michigan':'MI', 'Minnesota':'MN', 'Mississippi':'MS', 'Missouri':'MO','Montana':'MT', 'Nebraska':'NE', 'Nevada':'NV', 'New Hampshire':'NH', 'New Jersey':'NJ', 'New Mexico':'NM', 'New York':'NY', 'North Carolina':'NC', 'North Dakota':'ND', 'Ohio':'OH', 'Oklahoma':'OK','Oregon':'OR','Pennsylvania':'PA','Rhode Island':'RI', 'South Carolina':'SC', 'South Dakota':'SD', 'Tennessee':'TN','Texas':'TX', 'Utah':'UT','Vermont':'VT', 'Virginia':'VA', 'Washington':'WA', 'West Virginia':'WV','Wisconsin':'WI', 'Wyoming':'WY'})
#replace state name to match other dfs
poli_data['CountyName_x']=poli_data['county_name']+', '+poli_data['state_name'] #new column with County, ST to match format in other df

In [331]:
biden=[]

for i in poli_data['diff']:
    if i<0:
        biden.append(True)
    else:
        biden.append(False)
        
trump=[]

for j in poli_data['diff']:
    if j>0:
        trump.append(True)
    else:
        trump.append(False) #create binary variables for political affiliation

poli_data['biden']=biden
poli_data['trump']=trump

In [332]:
poli_data=pd.get_dummies(data=poli_data, columns=['biden', 'trump']) #creates dummies

In [333]:
poli_data=poli_data[['county_fips','CountyName_x','biden_True','trump_True']] #filters to necessary columns only

poli_data=poli_data[~poli_data['CountyName_x'].str.endswith('AK')] #removes entries from alaska 
poli_data['county_fips']=poli_data['county_fips'].astype(str)

In [334]:
poli_data=poli_data.rename(columns={'county_fips': 'fips'})

In [335]:
poli_data2=poli_data.append(poli_data)
poli_data3=poli_data2.append(poli_data)
poli_data4=poli_data3.append(poli_data)
poli_data5=poli_data4.append(poli_data)
poli_data6=poli_data5.append(poli_data)
poli_data7=poli_data6.append(poli_data)
poli_data8=poli_data7.append(poli_data)
poli_data9=poli_data8.append(poli_data)
poli_data10=poli_data9.append(poli_data)
poli_data11=poli_data10.append(poli_data)
poli_data12=poli_data11.append(poli_data)
poli_data13=poli_data12.append(poli_data)
poli_data14=poli_data13.append(poli_data)
poli_data15=poli_data14.append(poli_data)
poli_data16=poli_data15.append(poli_data)
poli_data17=poli_data16.append(poli_data)
poli_data18=poli_data17.append(poli_data)
poli_data19=poli_data18.append(poli_data)
poli_data20=poli_data19.append(poli_data)
poli_data21=poli_data20.append(poli_data)
poli_data22=poli_data21.append(poli_data)
poli_data23=poli_data22.append(poli_data)
poli_data24=poli_data23.append(poli_data)

poli_data=poli_data24.sort_values('CountyName_x')

In [336]:
poli_data=poli_data.reset_index()
demo_vax_ur=demo_vax_ur.reset_index()
poli_demo_ur_vax=pd.concat([poli_data,demo_vax_ur], axis=1)


In [338]:
poli_demo_ur_vax.loc[~poli_demo_ur_vax.index.duplicated(), :]

,index,fips,CountyName_x,biden_True,trump_True,level_0,Date,index,fips,Percent_Male,...,Series_Complete_Pop_Pct_UR_Equity,Series_Complete_5PlusPop_Pct_UR_Equity,Series_Complete_12PlusPop_Pct_UR_Equity,Series_Complete_18PlusPop_Pct_UR_Equity,Series_Complete_65PlusPop_Pct_UR_Equity,Census2019,Census2019_5PlusPop,Census2019_12PlusPop,Census2019_18PlusPop,Census2019_65PlusPop
0,2326,45001,"Abbeville County, SC",0,1,19,2020-01-31,2317,45001,48.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2326,45001,"Abbeville County, SC",0,1,8,2020-02-29,2317,45001,48.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2326,45001,"Abbeville County, SC",0,1,3,2020-03-31,2317,45001,48.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2326,45001,"Abbeville County, SC",0,1,7,2020-04-30,2317,45001,48.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2326,45001,"Abbeville County, SC",0,1,10,2020-05-31,2317,45001,48.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74683,2437,46137,"Ziebach County, SD",1,0,75383,2021-08-31,2428,46137,49.3,...,5.0,NaN,5.0,5.0,6.0,2756.0,NaN,2332.0,1997.0,NaN
74684,2437,46137,"Ziebach County, SD",1,0,75363,2021-09-30,2428,46137,49.3,...,5.0,NaN,5.0,5.0,6.0,2756.0,NaN,2332.0,1997.0,NaN
74685,2437,46137,"Ziebach County, SD",1,0,75365,2021-10-31,2428,46137,49.3,...,5.0,NaN,5.0,6.0,7.0,2756.0,NaN,2332.0,1997.0,NaN
74686,2437,46137,"Ziebach County, SD",1,0,75364,2021-11-30,2428,46137,49.3,...,5.0,NaN,6.0,6.0,7.0,2756.0,NaN,2332.0,1997.0,NaN


In [342]:
poli_demo_ur_vax=poli_demo_ur_vax.loc[:, poli_demo_ur_vax.columns.isin(['fips','Date','CountyName_x','biden_True','trump_True','Percent_Male', 'Percent_Female',
       'Percent_White', 'Percent_Black', 'Percent_AmerInd', 'Percent_Asian',
       'Percent_PacI','UR_rate','Completeness_pct','Series_Complete_18PlusPop_Pct','Booster_Doses_Vax_Pct','Booster_Doses_18Plus','Metro_status'])]

In [344]:
poli_demo_ur_vax.to_csv('/Users/caseycloutier/Desktop/Theses/Honors585 Paper Data/cleaned data/poli_demo_ur_vax.csv')

In [384]:
mask=pd.read_csv(path+'/mask-use-by-county.csv')

In [385]:
mask['COUNTYFP']=mask['COUNTYFP'].astype(str)
akfips=['2013','2016','2020','2050','2060','2068','2070','2090','2100','2105','2110','2122','2130','2150','2158','2164','2170','2180','2185','2188','2195','2198','2220','2230','2240','2261','2275','2282','2290']
mask=mask[mask.COUNTYFP.isin(akfips) == False]


In [386]:
mask=mask[~mask['COUNTYFP'].str.endswith('15005')]

In [388]:
mask2=mask.append(mask)
mask3=mask2.append(mask)
mask4=mask3.append(mask)
mask5=mask4.append(mask)
mask6=mask5.append(mask)
mask7=mask6.append(mask)
mask8=mask7.append(mask)
mask9=mask8.append(mask)
mask10=mask9.append(mask)
mask11=mask10.append(mask)
mask12=mask11.append(mask)
mask13=mask12.append(mask)
mask14=mask13.append(mask)
mask15=mask14.append(mask)
mask16=mask15.append(mask)
mask17=mask16.append(mask)
mask18=mask17.append(mask)
mask19=mask18.append(mask)
mask20=mask19.append(mask)
mask21=mask20.append(mask)
mask22=mask21.append(mask)
mask23=mask22.append(mask)
mask24=mask23.append(mask)

mask=mask24.sort_values('COUNTYFP')


In [389]:
poli_demo_ur_vax=poli_demo_ur_vax.loc[:,~poli_demo_ur_vax.columns.duplicated()]
mask=mask.rename(columns={'COUNTYFP': 'fips'})

In [390]:
poli_demo_ur_vax=poli_demo_ur_vax.sort_values('fips')

In [391]:
mask=mask.reset_index(drop=True)
poli_demo_ur_vax=poli_demo_ur_vax.reset_index(drop=True)
mask_poli_demo_ur_vax=pd.concat([mask,poli_demo_ur_vax], axis=1)

In [393]:
mask_poli_demo_ur_vax.to_csv('/Users/caseycloutier/Desktop/Theses/Honors585 Paper Data/cleaned data/mask_poli_demo_ur_vax')

In [422]:
gdp=pd.read_csv('/Users/caseycloutier/Desktop/Theses/Honors585 Paper Data/raw datasets/202021 GDP.csv', lineterminator='\n')

In [423]:
mask_poli_demo_ur_vax['Quarter']=mask_poli_demo_ur_vax['Date'].dt.to_period('Q')

In [424]:
mask_poli_demo_ur_vax['State']=[x[-2:] for x in mask_poli_demo_ur_vax['CountyName_x']]

In [425]:
gdp['State']=gdp['GeoName'].replace({'Alabama':'AL', 'Alaska':'AK', 'Arizona':'AZ', 'Arkansas':'AR', 'California':'CA', 'Colorado':'CO', 'Connecticut':'CT', 'Delaware':'DE', 'Florida':'FL', 'Georgia':'GA', 'Hawaii':'HI', 'Idaho':'ID', 'Illinois':'IL', 'Indiana':'IN', 'Iowa':'IA', 'Kansas':'KS', 'Kentucky':'KY','Louisiana':'LA', 'Maine':'ME', 'Maryland':'MD', 'Massachusetts':'MA', 'Michigan':'MI', 'Minnesota':'MN', 'Mississippi':'MS', 'Missouri':'MO','Montana':'MT', 'Nebraska':'NE', 'Nevada':'NV', 'New Hampshire':'NH', 'New Jersey':'NJ', 'New Mexico':'NM', 'New York':'NY', 'North Carolina':'NC', 'North Dakota':'ND', 'Ohio':'OH', 'Oklahoma':'OK','Oregon':'OR','Pennsylvania':'PA','Rhode Island':'RI', 'South Carolina':'SC', 'South Dakota':'SD', 'Tennessee':'TN','Texas':'TX', 'Utah':'UT','Vermont':'VT', 'Virginia':'VA', 'Washington':'WA', 'West Virginia':'WV','Wisconsin':'WI', 'Wyoming':'WY','District of Columbia':'DC'})

In [426]:
gdp=gdp.melt(id_vars=["State", "GeoName","GeoFips"], 
        var_name="Quarter", 
        value_name="Q_GDP")

In [427]:
gdp=gdp.sort_values('GeoName')

In [432]:
gdp['Quarter']=gdp['Quarter'].str.replace("[:\r]","")

<ipython-input-432-ad8c158b119f>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  gdp['Quarter']=gdp['Quarter'].str.replace("[:\r]","")


In [476]:
gdp['quarterstate']=gdp['Quarter'].astype(str)+gdp['State']
mask_poli_demo_ur_vax['quarterstate']=mask_poli_demo_ur_vax['Quarter'].astype(str)+mask_poli_demo_ur_vax['State']
gdp_mask_poli_demo_ur_vax=mask_poli_demo_ur_vax.join(gdp.set_index('quarterstate'), on='quarterstate', lsuffix = '_left', rsuffix = '_right')

In [477]:
gdp_mask_poli_demo_ur_vax=gdp_mask_poli_demo_ur_vax[['fips','CountyName_x','Date','Quarter_left','NEVER', 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS','biden_True', 'trump_True','Percent_Male', 'Percent_Female', 'Percent_White',
       'Percent_Black', 'Percent_AmerInd', 'Percent_Asian', 'Percent_PacI',
       'UR_rate', 'Completeness_pct', 'Series_Complete_18PlusPop_Pct','Q_GDP']]

In [481]:
gdp_mask_poli_demo_ur_vax['Quarter_left']=gdp_mask_poli_demo_ur_vax['Quarter_left'].astype(str)
gdp_mask_poli_demo_ur_vax=gdp_mask_poli_demo_ur_vax[~gdp_mask_poli_demo_ur_vax['Quarter_left'].str.startswith('2021Q4')]


,fips,fips,CountyName_x,Date,Quarter_left,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,...,Percent_Female,Percent_White,Percent_Black,Percent_AmerInd,Percent_Asian,Percent_PacI,UR_rate,Completeness_pct,Series_Complete_18PlusPop_Pct,Q_GDP
1,10001,10001,"Kent County, DE",2020-05-31,2020Q2,0.026,0.006,0.106,0.051,0.811,...,51.8,66.2,25.7,0.7,2.0,0.1,16.8,NaN,NaN,58989.4
2,10001,10001,"Kent County, DE",2020-11-30,2020Q4,0.026,0.006,0.106,0.051,0.811,...,51.8,66.2,25.7,0.7,2.0,0.1,4.9,NaN,NaN,63180.0
3,10001,10001,"Kent County, DE",2020-02-29,2020Q1,0.026,0.006,0.106,0.051,0.811,...,51.8,66.2,25.7,0.7,2.0,0.1,4.4,NaN,NaN,63227.4
4,10001,10001,"Kent County, DE",2020-03-31,2020Q1,0.026,0.006,0.106,0.051,0.811,...,51.8,66.2,25.7,0.7,2.0,0.1,5.6,NaN,NaN,63227.4
5,10001,10001,"Kent County, DE",2020-06-30,2020Q2,0.026,0.006,0.106,0.051,0.811,...,51.8,66.2,25.7,0.7,2.0,0.1,14,NaN,NaN,58989.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74682,9015,9015,"Windham County, CT",2020-03-31,2020Q1,0.022,0.041,0.082,0.137,0.718,...,50.5,89.2,2.3,0.6,1.4,0.0,3.9,NaN,NaN,245664.6
74683,9015,9015,"Windham County, CT",2020-01-31,2020Q1,0.022,0.041,0.082,0.137,0.718,...,50.5,89.2,2.3,0.6,1.4,0.0,4.8,NaN,NaN,245664.6
74684,9015,9015,"Windham County, CT",2020-09-30,2020Q3,0.022,0.041,0.082,0.137,0.718,...,50.5,89.2,2.3,0.6,1.4,0.0,6.9,NaN,NaN,237215.7
74685,9015,9015,"Windham County, CT",2021-04-30,2021Q2,0.022,0.041,0.082,0.137,0.718,...,50.5,89.2,2.3,0.6,1.4,0.0,7.4,98.4,40.2,244913.3


In [482]:
gdp_mask_poli_demo_ur_vax=gdp_mask_poli_demo_ur_vax.sort_values(['CountyName_x', 'Date'])
gdp_mask_poli_demo_ur_vax

,fips,fips,CountyName_x,Date,Quarter_left,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,...,Percent_Female,Percent_White,Percent_Black,Percent_AmerInd,Percent_Asian,Percent_PacI,UR_rate,Completeness_pct,Series_Complete_18PlusPop_Pct,Q_GDP
49951,45001,45001,"Abbeville County, SC",2020-01-31,2020Q1,0.117,0.102,0.243,0.166,0.371,...,51.5,69.9,27.6,0.1,0.3,0.0,3.6,NaN,NaN,211784.1
49956,45001,45001,"Abbeville County, SC",2020-02-29,2020Q1,0.117,0.102,0.243,0.166,0.371,...,51.5,69.9,27.6,0.1,0.3,0.0,3.7,NaN,NaN,211784.1
49960,45001,45001,"Abbeville County, SC",2020-03-31,2020Q1,0.117,0.102,0.243,0.166,0.371,...,51.5,69.9,27.6,0.1,0.3,0.0,3.8,NaN,NaN,211784.1
49944,45001,45001,"Abbeville County, SC",2020-04-30,2020Q2,0.117,0.102,0.243,0.166,0.371,...,51.5,69.9,27.6,0.1,0.3,0.0,13.8,NaN,NaN,194789.8
49945,45001,45001,"Abbeville County, SC",2020-05-31,2020Q2,0.117,0.102,0.243,0.166,0.371,...,51.5,69.9,27.6,0.1,0.3,0.0,13.9,NaN,NaN,194789.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,46137,46137,"Ziebach County, SD",2021-05-31,2021Q2,0.105,0.053,0.172,0.235,0.434,...,50.7,23.5,0.0,75.1,0.6,0.0,3.7,82.1,22.7,49069.7
52617,46137,46137,"Ziebach County, SD",2021-06-30,2021Q2,0.105,0.053,0.172,0.235,0.434,...,50.7,23.5,0.0,75.1,0.6,0.0,5.5,83.0,23.7,49069.7
52613,46137,46137,"Ziebach County, SD",2021-07-31,2021Q3,0.105,0.053,0.172,0.235,0.434,...,50.7,23.5,0.0,75.1,0.6,0.0,6.4,83.5,24.8,48943.9
52610,46137,46137,"Ziebach County, SD",2021-08-31,2021Q3,0.105,0.053,0.172,0.235,0.434,...,50.7,23.5,0.0,75.1,0.6,0.0,5.5,84.0,26.9,48943.9


In [483]:
gdp_mask_poli_demo_ur_vax.to_csv('/Users/caseycloutier/Desktop/Theses/Honors585 Paper Data/cleaned data/gdp_mask_poli_demo_ur_vax.csv')